In [ ]:
from sentence_transformers import SentenceTransformer

# Load the same model used for indexing
# Make sure this matches what generated your .npy files
embedding_model_name = "BAAI/bge-base-en-v1.5"
try:
    query_model = SentenceTransformer(embedding_model_name)
    print(f"Embedding model '{embedding_model_name}' loaded successfully.")
except Exception as e:
    print(f"Error loading embedding model: {e}")
    # Handle error: model might not be downloaded, or path is wrong
    query_model = None


def get_query_embedding(query_text: str):
    if not query_model:
        raise ValueError("Embedding model not loaded.")
    # The model expects a list of texts, even if it's just one
    #q_emb = enc.encode(question, normalize_embeddings=True).tolist()
    q_emb = query_model.encode(query_text, normalize_embeddings=True).tolist()
    return q_emb



In [11]:
%pip install groq
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from dotenv import load_dotenv
import os

load_dotenv()  # Now it should work


True

In [4]:
import re
from typing import List, Tuple, Set, Dict, Any, Optional

# --- Mappings (as you defined them) ---
COMPANY_TO_TICKER_HINTS = {
    "apple": "AAPL", "microsoft": "MSFT", "google": "GOOGL", "alphabet": "GOOGL",
    "amazon": "AMZN", "nvidia": "NVDA", "meta": "META", "tesla": "TSLA",
    "meta platforms": "META", "facebook": "META",  # Facebook is now Meta
    "berkshire hathaway": "BRK.B", "bank of america": "BAC", "boa": "BAC",
    "boeing": "BA", "coca cola": "KO", "coca-cola": "KO", "cola": "KO", "coke": "KO",
    "johnson & johnson": "JNJ", "j&j": "JNJ", "johnson and johnson": "JNJ",
    "procter & gamble": "PG", "p&g": "PG", "procter and gamble": "PG",
    "walmart": "WMT", "wal-mart": "WMT", "wally world": "WMT",
    "united parcel service": "UPS", "ups": "UPS", "parcel service": "UPS",
    "general electric": "GE", "ge": "GE",
    "ibm": "IBM", "international business machines": "IBM",
    "american express": "AXP", "amex": "AXP",
    "home depot": "HD", "hd": "HD",
    "mcdonald's": "MCD", "mcdonalds": "MCD", "mcd": "MCD", "mickey d's": "MCD",
    "jpmorgan": "JPM", "jp morgan": "JPM", "jpm": "JPM",
    "wells fargo": "WFC", "citigroup": "C", "citi": "C",
    "goldman sachs": "GS", "morgan stanley": "MS",
    "charles schwab": "SCHW", "schwab": "SCHW",
    "blackrock": "BLK", "black rock": "BLK",
    "s&p global": "SPGI", "sandp global": "SPGI",
    "moody's": "MCO", "moodys": "MCO",
    "intercontinental exchange": "ICE",
    "walt disney": "DIS", "disney": "DIS", "disney co": "DIS",
    "comcast": "CMCSA", "netflix": "NFLX",
    "verizon": "VZ", "at&t": "T", "att": "T",
    "t-mobile": "TMUS", "tmobile": "TMUS",
    "charter": "CHTR", "fox": "FOXA", "news corp": "NWSA",
    "honeywell": "HON", "union pacific": "UNP",
    "3m": "MMM", "caterpillar": "CAT", "cat": "CAT",
    "lockheed martin": "LMT", "raytheon": "RTX",
    "northrop grumman": "NOC", "northrop": "NOC",
    "illinois tool works": "ITW", "deere": "DE", "john deere": "DE",
    "fedex": "FDX", "fed ex": "FDX",
    "pepsico": "PEP", "pepsi": "PEP",
    "costco": "COST", "mondelez": "MDLZ", "colgate-palmolive": "CL", "colgate": "CL",
    "kimberly-clark": "KMB", "general mills": "GIS", "kraft heinz": "KHC",
    "altria": "MO", "philip morris": "PM", "pm": "PM",
    "exxon mobil": "XOM", "exxon": "XOM", "mobil": "XOM",
    "chevron": "CVX", "conocophillips": "COP", "conoco": "COP",
    "schlumberger": "SLB", "halliburton": "HAL",
    "eog resources": "EOG", "marathon petroleum": "MPC", "marathon": "MPC",
    "phillips 66": "PSX", "valero": "VLO",
    "kinder morgan": "KMI", "williams companies": "WMB",
    "devon energy": "DVN",
    "nextera energy": "NEE", "duke energy": "DUK",
    "southern company": "SO", "dominion energy": "D",
    "american electric power": "AEP", "exelon": "EXC",
    "sempra": "SRE", "xcel energy": "XEL",
    "public service enterprise group": "PEG",
    "consolidated edison": "ED", "entergy": "ETR", "firstenergy": "FE",
    "american tower": "AMT", "prologis": "PLD",
    "crown castle": "CCI", "equinix": "EQIX", "public storage": "PSA",
    "simon property": "SPG", "digital realty": "DLR",
    "welltower": "WELL", "realty income": "O",
    "alexandria real estate": "ARE", "avalonbay": "AVB",
    "equity residential": "EQR",
    "linde": "LIN", "air products": "APD",
    "sherwin-williams": "SHW", "sherwin williams": "SHW",
    "dow": "DOW", "dupont": "DD", "newmont": "NEM",
    "freeport-mcmoran": "FCX", "freeport": "FCX",
    "international paper": "IP", "ball": "BALL", "albemarle": "ALB",
    "vision 2020": "V2020", "vision2020": "V2020", "v2020": "V2020",
}



TICKER_TO_COMPANY_HINTS = {}
for name, ticker in COMPANY_TO_TICKER_HINTS.items():
    if ticker not in TICKER_TO_COMPANY_HINTS:
        TICKER_TO_COMPANY_HINTS[ticker] = name.title()
# Crucial check: Ensure BRK.A and V2020 are in TICKER_TO_COMPANY_HINTS
if "BRK.A" not in TICKER_TO_COMPANY_HINTS and "berkshire hathaway a" in COMPANY_TO_TICKER_HINTS :
     TICKER_TO_COMPANY_HINTS["BRK.A"] = COMPANY_TO_TICKER_HINTS["berkshire hathaway a"].title()
if "V2020" not in TICKER_TO_COMPANY_HINTS and "vision 2020" in COMPANY_TO_TICKER_HINTS:
     TICKER_TO_COMPANY_HINTS["V2020"] = COMPANY_TO_TICKER_HINTS["vision 2020"].title()


TICKER_REGEX = re.compile(r'\b([A-Z]{1,5}(\.[A-Z])?)\b')
YEAR_REGEX = re.compile(r'\b(19[7-9]\d|20\d{2})\b')
QUARTER_REGEX = re.compile(
    r'\b(?:Q([1-4])|'
    r'(?:Quarter\s*([1-4]))|'
    r'(1st|2nd|3rd|4th)\s*Quarter|'
    r'(first|second|third|fourth)\s*Quarter)\b',
    re.IGNORECASE
)
QUARTER_WORD_TO_NUM = {
    "1st": 1, "first": 1, "2nd": 2, "second": 2,
    "3rd": 3, "third": 3, "4th": 4, "fourth": 4,
}

class EntityType:
    TICKER = "ticker"; COMPANY = "company"; YEAR = "year"; QUARTER = "quarter"

ENTITY_PRIORITIES = {
    EntityType.TICKER: 0, EntityType.YEAR: 1,
    EntityType.QUARTER: 2, EntityType.COMPANY: 3
}

class QueryFocus:
    def __init__(self, ticker: str, year: Optional[int] = None, quarter: Optional[int] = None): # Removed original_segment for brevity
        self.ticker = ticker; self.year = year; self.quarter = quarter
    def __repr__(self):
        return f"QueryFocus(ticker='{self.ticker}', year={self.year}, quarter={self.quarter})"
    def __eq__(self, other):
        return isinstance(other, QueryFocus) and self.ticker == other.ticker and self.year == other.year and self.quarter == other.quarter
    def __hash__(self):
        return hash((self.ticker, self.year, self.quarter))

def _extract_entities_from_segment_text(segment_text: str) -> List[Dict[str, Any]]:
    """Extracts entities from a given text segment. Indices are relative to segment_text."""
    potential_entities: List[Dict[str, Any]] = []
    # Tickers
    for match in TICKER_REGEX.finditer(segment_text):
        ticker_candidate = match.group(1)
        if ticker_candidate in TICKER_TO_COMPANY_HINTS: # Check if it's a known ticker
            potential_entities.append({
                "text": ticker_candidate, "value": ticker_candidate, "type": EntityType.TICKER,
                "start": match.start(1), "end": match.end(1), "priority": ENTITY_PRIORITIES[EntityType.TICKER]
            })
    # Company names
    for cn_key in sorted(COMPANY_TO_TICKER_HINTS.keys(), key=len, reverse=True):
        ticker = COMPANY_TO_TICKER_HINTS[cn_key]
        pattern = r'\b' + re.escape(cn_key) + r'\b'
        for cmatch in re.finditer(pattern, segment_text, re.IGNORECASE):
            potential_entities.append({
                "text": cmatch.group(0), "value": ticker, "type": EntityType.COMPANY,
                "start": cmatch.start(0), "end": cmatch.end(0), "priority": ENTITY_PRIORITIES[EntityType.COMPANY]
            })
    # Years
    for ymatch in YEAR_REGEX.finditer(segment_text):
        year_str = ymatch.group(1)
        potential_entities.append({
            "text": year_str, "value": int(year_str), "type": EntityType.YEAR,
            "start": ymatch.start(1), "end": ymatch.end(1), "priority": ENTITY_PRIORITIES[EntityType.YEAR]
        })
    # Quarters
    for qmatch in QUARTER_REGEX.finditer(segment_text):
        q_val_str = qmatch.group(1) or qmatch.group(2) or qmatch.group(3) or qmatch.group(4)
        q_num = None
        if q_val_str: q_num = int(q_val_str) if q_val_str.isdigit() else QUARTER_WORD_TO_NUM.get(q_val_str.lower())
        if q_num:
            potential_entities.append({
                "text": qmatch.group(0), "value": q_num, "type": EntityType.QUARTER,
                "start": qmatch.start(0), "end": qmatch.end(0), "priority": ENTITY_PRIORITIES[EntityType.QUARTER]
            })
    return potential_entities

# Sentence delimiters
SENTENCE_DELIMITERS_REGEX = re.compile(r'[.?!]')
CLAUSE_DELIMITERS_REGEX = re.compile(r'\s+(?:and|or|but)\s+|,', re.IGNORECASE)

def extract_structured_metadata(query: str) -> Tuple[List[QueryFocus], str]:
    all_query_focuses_set: Set[QueryFocus] = set()
    
    # --- Global Entity Extraction (for final query cleaning and global context) ---
    glob_potential_entities = _extract_entities_from_segment_text(query)
    glob_potential_entities.sort(key=lambda x: (x["start"], x["priority"], -(x["end"] - x["start"])))
    
    all_extracted_entities_globally: List[Dict[str, Any]] = []
    _last_covered_idx = -1
    for entity in glob_potential_entities:
        if entity["start"] >= _last_covered_idx:
            all_extracted_entities_globally.append(entity)
            _last_covered_idx = entity["end"]
    
    # --- Determine "Global Default" Context (only if unique across whole query) ---
    global_years_map = {}
    global_quarters_map = {}
    for entity in all_extracted_entities_globally:
        if entity["type"] == EntityType.YEAR: global_years_map[entity["value"]] = global_years_map.get(entity["value"], 0) + 1
        elif entity["type"] == EntityType.QUARTER: global_quarters_map[entity["value"]] = global_quarters_map.get(entity["value"], 0) + 1
            
    default_year = list(global_years_map.keys())[0] if len(global_years_map) == 1 else None
    default_quarter = list(global_quarters_map.keys())[0] if len(global_quarters_map) == 1 else None
    
    # --- Hierarchical Segmentation ---
    query_segments = []
    initial_sentences = SENTENCE_DELIMITERS_REGEX.split(query)
    for sentence_text in initial_sentences:
        if not sentence_text.strip(): continue
        clauses_text = []
        last_clause_split_end = 0
        for match in CLAUSE_DELIMITERS_REGEX.finditer(sentence_text):
            clauses_text.append(sentence_text[last_clause_split_end:match.start()].strip())
            last_clause_split_end = match.end()
        clauses_text.append(sentence_text[last_clause_split_end:].strip())
        query_segments.extend([c for c in clauses_text if c])
    
    if not query_segments and query.strip(): # If no delimiters found, whole query is one segment
        query_segments.append(query.strip())

    # --- Process Each Segment for Pairing ---
    for segment_text in query_segments:
        if not segment_text.strip(): continue

        segment_potential_entities = _extract_entities_from_segment_text(segment_text)
        segment_potential_entities.sort(key=lambda x: (x["start"], x["priority"], -(x["end"] - x["start"])))

        segment_selected_entities: List[Dict[str, Any]] = []
        _last_seg_idx = -1
        for entity in segment_potential_entities:
            if entity["start"] >= _last_seg_idx:
                segment_selected_entities.append(entity)
                _last_seg_idx = entity["end"]
        
        seg_stocks = [e for e in segment_selected_entities if e["type"] in (EntityType.TICKER, EntityType.COMPANY)]
        seg_years = sorted([e for e in segment_selected_entities if e["type"] == EntityType.YEAR], key=lambda y: y["start"])
        seg_quarters = sorted([e for e in segment_selected_entities if e["type"] == EntityType.QUARTER], key=lambda q: q["start"])

        if not seg_stocks: continue

        for stock_entity in seg_stocks:
            current_ticker = stock_entity["value"]
            assigned_year: Optional[int] = None
            assigned_quarter: Optional[int] = None

            if seg_years:
                assigned_year = min(seg_years, key=lambda y: abs(y["start"] - stock_entity["start"]))["value"]
            if seg_quarters:
                assigned_quarter = min(seg_quarters, key=lambda q: abs(q["start"] - stock_entity["start"]))["value"]
            
            # Apply defaults with more conservative logic
            if assigned_year is None:
                # Only apply default_year if segment has NO quarter, or if it has a quarter but default_year is "strong"
                if assigned_quarter is None and default_year is not None: # No time info in segment
                    assigned_year = default_year
                elif assigned_quarter is not None and default_year is not None: # Segment has Q, missing Y
                     assigned_year = default_year # This allows Tesla 2023 Q1 if default_year=2023
                     # To prevent Tesla 2023 Q1: default_year should only apply if segment has NO quarter.
                     # Let's refine:
                     # if assigned_quarter is None and default_year is not None:
                     #     assigned_year = default_year

            # More conservative default year:
            if assigned_year is None and assigned_quarter is None and default_year is not None:
                 assigned_year = default_year
            # If segment has a quarter but no year, only then try default year.
            elif assigned_year is None and assigned_quarter is not None and default_year is not None:
                 assigned_year = default_year


            if assigned_quarter is None and default_quarter is not None:
                # Apply default quarter if there's some year context or it's the only thing
                if assigned_year is not None: # Year context exists for this stock
                    assigned_quarter = default_quarter
                elif default_year is None : # No year context at all, but a lone default quarter
                    assigned_quarter = default_quarter


            all_query_focuses_set.add(QueryFocus(ticker=current_ticker, year=assigned_year, quarter=assigned_quarter))

    # --- Final Query Modification ---
    modified_query_parts = []
    current_pos = 0
    for entity in all_extracted_entities_globally: # Use globally resolved entities
        if entity["start"] > current_pos:
            modified_query_parts.append(query[current_pos:entity["start"]])
        
        end_skip = entity["end"]
        if entity["type"] in (EntityType.TICKER, EntityType.COMPANY) and not entity["text"].lower().endswith("'s"):
            if entity["end"] + 1 < len(query) and query[entity["end"]] == "'" and query[entity["end"]+1].lower() == "s":
                end_skip = entity["end"] + 2
        current_pos = end_skip

    if current_pos < len(query): modified_query_parts.append(query[current_pos:])
    modified_query = ' '.join("".join(modified_query_parts).split()).strip()

    return sorted(list(all_query_focuses_set), key=lambda qf: (
        qf.ticker, qf.year or -1, qf.quarter or -1
    )), modified_query

#groq intilizing
import groq
import json
import os
groq.api_key = os.getenv("GROQ_API_KEY")
# Ensure the environment variable is set
if not groq.api_key:
    raise ValueError("GROQ_API_KEY environment variable is not set.")
# --- Functionality Test ---
try:
    groq_client = groq.Client()
    print("Groq client initialized successfully.")
except Exception as e:
    print(f"Error initializing Groq client: {e}")
    # Handle error: client might not be initialized properly
    groq_client = None


LLM_MODEL_NAME = "llama3-8b-8192" # Or "mixtral-8x7b-32768", "llama3-70b-8192" etc.

def format_rule_based_output_for_prompt(focuses: List[QueryFocus]) -> str:
    if not focuses:
        return "Rule-based extraction found no specific company/ticker focuses."
    
    formatted_str = "Rule-Based Extraction Results:\n"
    for i, focus in enumerate(focuses):
        year_str = str(focus.year) if focus.year is not None else "Not specified"
        quarter_str = "Q" + str(focus.quarter) if focus.quarter is not None else "Not specified"
        formatted_str += f"- Focus {i+1}: Ticker={focus.ticker}, Year={year_str}, Quarter={quarter_str}\n"
    return formatted_str.strip()

def refine_with_llm(original_query: str, rule_based_focuses: List[QueryFocus]) -> List[QueryFocus]:
    if not groq_client:
        print("Groq client not initialized. Returning rule-based results.")
        return rule_based_focuses

    rule_based_output_str = format_rule_based_output_for_prompt(rule_based_focuses)

    system_prompt = """
You are an expert financial data analyst assistant. Your task is to review a user's query and a preliminary rule-based extraction of company tickers, years, and quarters.
Your goal is to produce a corrected list of (Ticker, Year, Quarter) focuses that accurately reflect all distinct requests for financial data implied by the user's query.

Instructions:
1. Analyze the "Original Query" to understand the user's full intent, paying close attention to context.
2. Review the "Rule-Based Extraction Results". This is an initial attempt and may contain errors, omissions, or irrelevant focuses.

IMPORTANT - EVALUATING AND REFINING FOCUSES:
- Your primary role is to:
    a) CORRECT errors in the pairing of (Ticker, Year, Quarter) within each valid focus.
    b) ADD any focuses for financial data requests COMPLETELY MISSED by the rule-based system.
    c) REMOVE focuses from the rule-based results if they correspond to a company/ticker mention that, in the context of the *entire query*, is CLEARLY NOT a request for that company's financial data. For example:
        - If "Apple" is mentioned in the context of eating fruit, a rule-based focus for "AAPL" should be removed.
        - If a company is mentioned as a source of data (e.g., "Data provided by FactSet"), a focus for FactSet's own financials should be removed unless explicitly requested.
        - If a company name is part of a general phrase or a different entity (e.g., "Amazon River tour"), a focus for "AMZN" should be removed.
- However, if a company/ticker is mentioned and the context suggests a potential interest in its financial data or performance, even if part of a complex sentence, you should AIM TO PRESERVE and CORRECT its focus rather than discarding it. For example, if the query is "Info on Apple in 2023 Q1 and also Google news", and the rule-based system provides focuses for both Apple and Google, your output MUST include focuses for both, refining their year/quarter as needed.

4. If a year or quarter is not specified or implied for a particular company's valid financial data request, use `null` for that field.
5. Ensure years and quarters are correctly associated with their respective tickers for each valid request.
6. If the rule-based extraction is already perfect for all valid financial data requests, your output should be identical to it.

Output Format:
Return ONLY a valid JSON list of objects. Each object MUST have: "ticker" (string), "year" (integer or null), "quarter" (integer 1-4 or null).
Do NOT include any other text.
Example 1:
Query: "Apple Q1 2022 and Google Q2 2023"
Expected JSON:
[
  {"ticker": "AAPL", "year": 2022, "quarter": 1},
  {"ticker": "GOOGL", "year": 2023, "quarter": 2}
]

Example 2:
Query: "Microsoft and Nvidia performance in 2023 Q4"
Expected JSON:
[
  {"ticker": "MSFT", "year": 2023, "quarter": 4},
  {"ticker": "NVDA", "year": 2023, "quarter": 4}
]

Example 3:
Query: "Tesla sales for Q1 this year. Also Amazon results."
(Assume "this year" means current year, e.g., 2024, for the LLM to potentially infer or for you to pre-process)
Expected JSON (if LLM infers current year for Tesla, or you add it):
[
  {"ticker": "TSLA", "year": 2024, "quarter": 1},
  {"ticker": "AMZN", "year": null, "quarter": null}
]
(If LLM doesn't infer year for Tesla)
[
  {"ticker": "TSLA", "year": null, "quarter": 1},
  {"ticker": "AMZN", "year": null, "quarter": null}
]
Example (Contextual Discard):
Query: "I like to eat an apple a day. What about Microsoft's Q1 earnings?"
Rule-Based: [{"ticker": "AAPL", "year": null, "quarter": null}, {"ticker": "MSFT", "year": null, "quarter": 1}]
Expected JSON:
[
  {"ticker": "MSFT", "year": null, "quarter": 1}
]
// Note: AAPL focus removed due to fruit context.

Example (Preserving Multiple Valid Requests):
Query: "Company Vision 2020 report for 2020 Q1 and Apple news."
Rule-Based: [{"ticker": "V2020", "year": 2020, "quarter": 1}, {"ticker": "AAPL", "year": 2020, "quarter": 1}]
Expected JSON (LLM should preserve both, possibly refining AAPL's year/quarter if context suggests different):
[
  {"ticker": "V2020", "year": 2020, "quarter": 1},
  {"ticker": "AAPL", "year": 2020, "quarter": 1}
]
// Or, if LLM interprets "Apple news" as more general:
// [
//   {"ticker": "V2020", "year": 2020, "quarter": 1},
//   {"ticker": "AAPL", "year": null, "quarter": null}
// ]
// The key is both V2020 and AAPL are considered subjects.
"""

    user_content = f"""
Original Query:
"{original_query}"

Rule-Based Extraction Results:
{rule_based_output_str}

Task:
Provide the "Corrected List of Focuses" as a JSON list of objects based on the instructions.
JSON list:
"""

    try:
        chat_completion = groq_client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_content,
                }
            ],
            model=LLM_MODEL_NAME,
            temperature=0.1, # Lower temperature for more deterministic, factual output
            max_tokens=1024, # Adjust as needed
            # response_format={"type": "json_object"} # If supported by Groq and model, helps ensure JSON
        )

        response_content = chat_completion.choices[0].message.content
        
        # LLMs sometimes add markdown backticks around JSON, try to strip them
        if response_content.startswith("```json"):
            response_content = response_content[7:]
            if response_content.endswith("```"):
                response_content = response_content[:-3]
        response_content = response_content.strip()

        # print(f"LLM Raw Response: {response_content}") # For debugging

        llm_focuses_json = json.loads(response_content)
        
        # Validate and convert to QueryFocus objects
        refined_focuses = []
        if isinstance(llm_focuses_json, list):
            for item in llm_focuses_json:
                if isinstance(item, dict) and "ticker" in item:
                    refined_focuses.append(
                        QueryFocus(
                            ticker=item.get("ticker"),
                            year=item.get("year"), # Handles null
                            quarter=item.get("quarter") # Handles null
                        )
                    )
                else:
                    print(f"Warning: LLM returned an invalid item in the list: {item}")
            return sorted(refined_focuses, key=lambda qf: (
                qf.ticker, qf.year or -1, qf.quarter or -1
            ))
        else:
            print(f"Warning: LLM did not return a list as expected. Response: {llm_focuses_json}")
            return rule_based_focuses # Fallback

    except json.JSONDecodeError as e:
        print(f"Error decoding LLM JSON response: {e}")
        print(f"LLM Raw Response was: {response_content}")
        return rule_based_focuses # Fallback to rule-based if LLM output is not valid JSON
    except Exception as e:
        print(f"An error occurred during LLM refinement: {e}")
        return rule_based_focuses # Fallback

# --- Main processing pipeline ---
def process_query_with_llm_refinement(query: str) -> Tuple[List[QueryFocus], str]:
    # 1. Rule-based extraction
    rule_based_focuses, modified_query_by_rules = extract_structured_metadata(query)
    
    # 2. LLM refinement
    # We pass the original query and the rule_based_focuses to the LLM.
    # The modified_query_by_rules is mainly for if you wanted to show the user what's "left"
    # or for further processing that doesn't need the entities. The LLM works best with the original query.
    
    if groq_client: # Only call LLM if client is available
        print(f"\n--- Calling LLM for query: \"{query}\" ---")
        print(f"Rule-based output: {rule_based_focuses}")
        refined_focuses = refine_with_llm(query, rule_based_focuses)
        print(f"LLM refined output: {refined_focuses}")
        # The modified query string should still come from the rule-based system,
        # as the LLM is focused on correcting the structured (Ticker, Year, Quarter) data.
        # If the LLM adds/removes entities, the modified_query_by_rules might become inconsistent
        # if not re-calculated. For now, we'll use the rule-based modified_query.
        return refined_focuses, modified_query_by_rules
    else:
        return rule_based_focuses, modified_query_by_rules





Groq client initialized successfully.


In [10]:
from qdrant_client import QdrantClient
import numpy as np
from qdrant_client.models import Filter, FieldCondition, MatchValue


QDRANT_URL_LOCAL = "http://localhost:6333"
COLLECTION_NAME_LOCAL = "financial_sp500_local_final_v2" # Your collection name
client = QdrantClient(url=QDRANT_URL_LOCAL)


# ------- 1. Focus  ->  Filter --------------------------------------------
def focus_to_filter(focus: QueryFocus) -> Filter:
    """
    Convert a single QueryFocus(TKR, year?, qtr?) to a Qdrant payload Filter.
    Any attribute that is None is simply omitted, giving you the usual
    ‘match everything’ behaviour for that field.
    """
    must_conditions: List[FieldCondition] = [
        FieldCondition(
            key="ticker",              # payload key in Qdrant
            match=MatchValue(value=focus.ticker)
        )
    ]

    if focus.year is not None:
        must_conditions.append(
            FieldCondition(
                key="year",
                match=MatchValue(value=focus.year)
            )
        )

    if focus.quarter is not None:
        must_conditions.append(
            FieldCondition(
                key="quarter",
                match=MatchValue(value=focus.quarter)
            )
        )

    return Filter(must=must_conditions)   # “AND” all present fields


# ------- 2. Search with OR-of-focuses -------------------------------------
# ------- 3. Separate search for every focus -------------------------------
from collections import OrderedDict
from dataclasses import asdict           # only for pretty printing

def search_qdrant_per_focus(
    query_vector,
    focuses: List[QueryFocus],
    k: int = 10,
    with_payload: bool = True,
    with_vectors: bool = False,
    client: QdrantClient = client,
    collection_name: str = COLLECTION_NAME_LOCAL,
):
    """
    Run *one* similarity search for *each* QueryFocus so the results stay
    strictly separated (e.g., Google 2022 vs. Apple 2021).

    Returns
    -------
    OrderedDict[QueryFocus, list[qdrant_client.http.models.ScoredPoint]]
        Keys preserve the input focus order.  Each value is the list of hits
        for that focus.
    """
    # Convert numpy vector once
    vec = query_vector.tolist() if hasattr(query_vector, "tolist") else query_vector

    results = OrderedDict()
    for focus in focuses:
        res = client.search(
            collection_name=collection_name,
            query_vector=vec,
            limit=k,
            query_filter=focus_to_filter(focus),
            with_payload=with_payload,
            with_vectors=with_vectors,
            timeout=10
        )
        results[focus] = res
    return results



In [19]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models
import numpy as np, pprint, textwrap

MODEL_NAME = "BAAI/bge-base-en-v1.5"
COLLECTION  = "financial_sp500_local_final_v2"
TOP_K       = 5

# 1. Connect and load encoder
enc = SentenceTransformer(MODEL_NAME, device="cpu")   # or "cuda"
cli = QdrantClient(url="http://localhost:6333")

# 2. Ad-hoc query
query = "What were the risk factors for Apple in 2024 from the annual report?"
focuses, _ = process_query_with_llm_refinement(query)
embed = enc.encode(query, normalize_embeddings=True).tolist()


hits = search_qdrant_per_focus(
    query_vector=embed,
    focuses=focuses,
    k=TOP_K,
    client=cli,
    collection_name=COLLECTION
)

for focus, hits_list in hits.items(): # Renamed 'hits' to 'hits_list' to avoid conflict with outer scope 'hits'
    print("Focus:", focus) # Changed from asdict(focus) to focus
    for h in hits_list: # Use hits_list here
        print(f"  Score: {h.score:.4f}, ID: {h.id}, Payload: {h.payload}")
        if 'text' in h.payload:
            print("  Text:", textwrap.fill(h.payload['text'], width=80))



--- Calling LLM for query: "What were the risk factors for Apple in 2024 from the annual report?" ---
Rule-based output: [QueryFocus(ticker='AAPL', year=2024, quarter=None)]
Error decoding LLM JSON response: Extra data: line 5 column 1 (char 57)
LLM Raw Response was: [
  {"ticker": "AAPL", "year": 2024, "quarter": null}
]

Note: The rule-based extraction result is already correct, so the corrected list is identical to the original result.
LLM refined output: [QueryFocus(ticker='AAPL', year=2024, quarter=None)]


C:\Users\Serban\AppData\Local\Temp\ipykernel_28492\3240241699.py:73: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  res = client.search(


Focus: QueryFocus(ticker='AAPL', year=2024, quarter=None)
  Score: 0.7302, ID: 9dfe42f6-2887-499e-9fe8-b56a5fb2d9b0, Payload: {'source_type': 'sec_filing', 'filing_category': '10q', 'original_file_name': '2024_2024-08-02.json', 'ticker': 'AAPL', 'year': 2024, 'cik': '0000320193', 'date': '2024-08-02', 'filing_type': '10-Q', 'item': '1A', 'section_name': 'Risk Factors', 'chunk_sequence_in_section': 0, 'global_chunk_id': 'sec_4010304', 'chunk_text': 'Risk Factors The Company\'s business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described in Part I, Item 1A of the 2023 Form 10-K and Part II, Item 1A of the Form 10-Q for the quarter ended March 30, 2024 (the "second quarter 2024 Form 10-Q"), in each case under the heading "Risk Factors." When any one or more of these risks materialize from time to time, the Company\'s business, reputation, results of operations, financi

In [ ]:
# (Assuming your previous code for query_model, client, get_query_embedding, search_qdrant is present)

def format_retrieved_context(search_results, max_context_chars=15000):
    """
    Formats the search results from Qdrant into a single string
    to be used as context for the LLM.
    Also returns a list of source metadata for citation.
    """
    context_parts = []
    sources_metadata = []
    current_chars = 0

    if not search_results:
        return "", []

    for i, hit in enumerate(search_results):
        payload = hit.payload
        if payload:
            chunk_text = payload.get("chunk_text", "")
            
            # Estimate if adding this chunk will exceed the character limit
            if current_chars + len(chunk_text) > max_context_chars and context_parts:
                print(f"Context character limit ({max_context_chars}) reached. Stopping context assembly.")
                break # Stop adding more chunks if limit is close

            source_info = f"Source {i+1}:\n"
            source_info += f"  Ticker: {payload.get('ticker', 'N/A')}\n"
            source_info += f"  Year: {payload.get('year', 'N/A')}\n"
            source_info += f"  Filing: {payload.get('filing_type', payload.get('filing_category', 'N/A'))}\n"
            if payload.get('source_type') == 'sec_filing':  #need a check to see if that is what is called
                source_info += f"  Section: {payload.get('section_name', 'N/A')}\n"
                source_info += f"  Item: {payload.get('item', 'N/A')}\n"
            elif payload.get('source_type') == 'earnings_transcript':
                source_info += f"  Quarter: {payload.get('quarter', 'N/A')}\n"
                source_info += f"  Speaker: {payload.get('turn_speaker_simple', 'N/A')}\n"
                source_info += f"  Call Section: {payload.get('turn_section', 'N/A')}\n"
            # source_info += f"  Original File: {payload.get('original_file_name', 'N/A')}\n" # Optional
            # source_info += f"  (Qdrant Score: {hit.score:.4f})\n" # Optional, for debugging relevance

            context_parts.append(source_info)
            context_parts.append(f"  Content: {chunk_text}\n---\n")
            
            sources_metadata.append(payload) # Store the full payload for more detailed citation if needed
            current_chars += len(source_info) + len(chunk_text) + 5 # Rough estimate for newlines etc.
        
    return "".join(context_parts), sources_metadata

In [ ]:
from openai import OpenAI

# --- LLM Configuration ---
# IMPORTANT: Set your OpenAI API key as an environment variable:
# export OPENAI_API_KEY="your_api_key_here"
# Or, pass it directly: client_openai = OpenAI(api_key="your_api_key_here")
try:
    llm_client = OpenAI() # Reads API key from environment variable OPENAI_API_KEY
    LLM_MODEL_NAME = "gpt-3.5-turbo" # Or "gpt-4", "gpt-4-turbo-preview", etc.
    print(f"OpenAI client initialized for model: {LLM_MODEL_NAME}")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    print("Please ensure the 'openai' library is installed and your API key is set.")
    llm_client = None


def get_llm_response(query_text: str, context_string: str):
    if not llm_client:
        raise ValueError("LLM client not initialized.")

    system_prompt = "You are a helpful financial analyst assistant. Answer the user's question based ONLY on the provided context. If the information is not in the context, say you don't know or that the context doesn't provide the answer. Be concise and cite the sources if specific information is used."
    
    user_message = f"""
Context from financial documents:
---
{context_string}
---
User Question: {query_text}

Answer:
"""
    try:
        completion = llm_client.chat.completions.create(
            model=LLM_MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.2 # Lower temperature for more factual, less creative answers
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error getting response from LLM: {e}")
        return "Error: Could not get a response from the LLM."

In [ ]:
def answer_query_with_rag(user_query: str, top_k_retrieval=5):
    print(f"\nProcessing query: '{user_query}'")

    # 1. Embed the query
    print("Embedding user query...")
    try:
        # For BGE models, add the recommended prefix for retrieval queries
        if "bge-" in embedding_model_name.lower(): # embedding_model_name from your setup
            query_for_embedding = f"Represent this sentence for searching relevant passages: {user_query}"
        else:
            query_for_embedding = user_query
        
        query_vector = get_query_embedding(query_for_embedding) # Use your embedding function
        print(f"Query embedded (first 5 dims): {query_vector[:5]}...")
    except Exception as e:
        print(f"Error embedding query: {e}")
        return "Could not process the query due to an embedding error."

    # 2. Search Qdrant for relevant chunks
    print(f"Searching Qdrant for top {top_k_retrieval} relevant chunks...")
    try:
        search_results = search_qdrant(query_vector, top_k=top_k_retrieval) # Use your Qdrant search
        if not search_results:
            print("No relevant documents found in Qdrant.")
            return "I could not find relevant information in the documents to answer your question."
        print(f"Found {len(search_results)} potential chunks.")
    except Exception as e:
        print(f"Error searching Qdrant: {e}")
        return "Could not process the query due to a database search error."

    # 3. Format the retrieved context
    print("Formatting context for LLM...")
    context_string, sources_metadata = format_retrieved_context(search_results)
    if not context_string:
        print("No context could be formatted (perhaps chunks were empty or too large).")
        return "I found some documents, but could not prepare them to answer your question."
    
    # print("\n--- Context being sent to LLM ---")
    # print(context_string[:1000] + "..." if len(context_string) > 1000 else context_string)
    # print("--- End of Context ---")

    # 4. Get response from LLM
    print("Getting response from LLM...")
    try:
        llm_answer = get_llm_response(user_query, context_string)
    except Exception as e:
        print(f"Error in LLM call: {e}")
        return "An error occurred while trying to generate an answer with the LLM."
        
    # 5. Return the answer (and optionally sources)
    # You can enhance this to return structured output
    print("\n--- RAG Process Complete ---")
    return llm_answer, sources_metadata

# --- Example Usage ---
if __name__ == "__main__":
    if not LIBRARIES_AVAILABLE or not query_model or not client or not llm_client:
        print("Exiting: One or more essential components (libraries, models, clients) failed to initialize.")
    else:
        # Test queries
        queries = [
            "What were Agilent's main business segments in 2018?",
            "What did Agilent say about the COVID-19 impact in early 2020?",
            "Summarize the key risks for a major tech company in their 2023 10-K.",
            "What was Apple's revenue in their latest reported quarter for which data exists?" # This requires up-to-date data
        ]

        for q in queries:
            answer, sources = answer_query_with_rag(q, top_k_retrieval=3) # Get 3 chunks for context
            print(f"\n\nQuery: {q}")
            print(f"LLM Answer:\n{answer}")
            
            # print("\nSources Used (Payloads):")
            # for i, src_meta in enumerate(sources):
            #     print(f"  Source {i+1} Ticker: {src_meta.get('ticker')}, File: {src_meta.get('original_file_name')}, Section: {src_meta.get('section_name', src_meta.get('turn_section'))}")
            print("-" * 50)